In [1]:
import os 
from dotenv import load_dotenv
load_dotenv('../../.env')

model=os.environ.get('model')
print(model)

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get('AZURE_API_BASE')
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get('AZURE_API_KEY')
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ.get('AZURE_API_VERSION')

azure/gpt-4


In [2]:
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from typing import List, Dict, Any

In [3]:
class TravelCrew:
    def __init__(self):
        # Create specialized travel agents
        self.flight_agent = Agent(
            role='Flight Expert',
            goal='Help users with flight-related queries and bookings',
            backstory=dedent("""
                You are an expert in flight bookings and airline operations. 
                You have extensive knowledge of airlines, routes, and booking processes.
            """),
            verbose=True,
            llm=model,
        )

        self.train_agent = Agent(
            role='Train Expert',
            goal='Help users with train-related queries and bookings',
            backstory=dedent("""
                You are an expert in train travel and railway systems.
                You have deep knowledge of train routes, schedules, and booking procedures.
            """),
            verbose=True,
            llm=model,
        )

        self.travel_assistant = Agent(
            role='General Travel Assistant',
            goal='Help with general travel planning and queries',
            backstory=dedent("""
                You are a knowledgeable travel assistant with expertise in 
                various aspects of travel planning including accommodations,
                local transportation, and tourist activities.
            """),
            verbose=True,
            llm=model,
        )

In [4]:
class FinanceCrew:
    def __init__(self):
        # Create specialized finance agents
        self.stock_agent = Agent(
            role='Stock Market Expert',
            goal='Provide stock market analysis and investment advice',
            backstory=dedent("""
                You are a seasoned stock market analyst with extensive experience
                in equity markets and investment strategies.
            """),
            verbose=True,
            llm=model,
        )

        self.mutual_fund_agent = Agent(
            role='Mutual Fund Expert',
            goal='Provide mutual fund investment advice',
            backstory=dedent("""
                You are an expert in mutual funds with deep knowledge of 
                different fund types, risk assessment, and portfolio management.
            """),
            verbose=True,
            llm=model,
        )

        self.finance_assistant = Agent(
            role='General Finance Assistant',
            goal='Help with general financial queries',
            backstory=dedent("""
                You are a knowledgeable financial advisor who can help with
                basic financial planning and general money management questions.
            """),
            verbose=True,
            llm=model,
        )

In [5]:
class SupervisorSystem:
    def __init__(self):
        # Initialize specialized crews
        self.travel_crew = TravelCrew()
        self.finance_crew = FinanceCrew()

        # Create supervisor agents
        self.travel_supervisor = Agent(
            role='Travel Supervisor',
            goal='Route and manage travel-related queries',
            backstory=dedent("""
                You are a supervisor responsible for analyzing travel-related queries
                and directing them to the appropriate specialized agent.
            """),
            verbose=True,
            llm=model,
        )

        self.finance_supervisor = Agent(
            role='Finance Supervisor',
            goal='Route and manage finance-related queries',
            backstory=dedent("""
                You are a supervisor responsible for analyzing finance-related queries
                and directing them to the appropriate specialized agent.
            """),
            verbose=True,
            llm=model,
        )

        self.main_supervisor = Agent(
            role='Main Supervisor',
            goal='Route queries to appropriate department',
            backstory=dedent("""
                You are the main supervisor responsible for analyzing incoming queries
                and routing them to either the travel or finance department.
            """),
            verbose=True,
            llm=model,
        )

    def _create_travel_tasks(self, query: str) -> List[Task]:
        # Analyze query to determine specific travel agent
        if "flight" in query.lower() or "airline" in query.lower():
            agent = self.travel_crew.flight_agent
        elif "train" in query.lower() or "railway" in query.lower():
            agent = self.travel_crew.train_agent
        else:
            agent = self.travel_crew.travel_assistant

        return [
            Task(
                description=f"Process travel query: {query}",
                expected_output="A satisfactory resolution of the query",
                agent=agent
            )
        ]

    def _create_finance_tasks(self, query: str) -> List[Task]:
        # Analyze query to determine specific finance agent
        if "stock" in query.lower() or "shares" in query.lower():
            agent = self.finance_crew.stock_agent
        elif "mutual fund" in query.lower() or "mf" in query.lower():
            agent = self.finance_crew.mutual_fund_agent
        else:
            agent = self.finance_crew.finance_assistant

        return [
            Task(
                description=f"Process finance query: {query}",
                expected_output="A satisfactory resolution of the query",
                agent=agent
            )
        ]

    def process_query(self, query: str) -> str:
        # First, determine if query is travel or finance related
        travel_keywords = {"travel", "flight", "train", "hotel", "booking", "trip", "vacation"}
        finance_keywords = {"money", "invest", "stock", "fund", "financial", "market"}
        
        query_lower = query.lower()
        is_travel = any(keyword in query_lower for keyword in travel_keywords)
        is_finance = any(keyword in query_lower for keyword in finance_keywords)

        # Create appropriate tasks based on query type
        if is_travel:
            tasks = self._create_travel_tasks(query)
            supervisor = self.travel_supervisor
        elif is_finance:
            tasks = self._create_finance_tasks(query)
            supervisor = self.finance_supervisor
        else:
            # Default to travel for unclassified queries
            tasks = self._create_travel_tasks(query)
            supervisor = self.travel_supervisor

        # Create crew with appropriate supervisor and tasks
        crew = Crew(
            agents=[supervisor],
            tasks=tasks,
            process=Process.sequential
        )

        # Execute the tasks
        result = crew.kickoff()
        return result

In [6]:
def main():
    # Initialize the system
    system = SupervisorSystem()

    # Example queries
    example_queries = [
        "I need to book a flight from New York to London",
        "What are the best mutual funds to invest in?",
        "Help me plan a vacation in Europe",
        "Should I invest in tech stocks?",
        "I need a train ticket to Boston"
    ]

    # Process each query
    for query in example_queries:
        print(f"\nProcessing query: {query}")
        result = system.process_query(query)
        print(f"Response: {result}")

In [7]:
if __name__ == "__main__":
    main()


Processing query: I need to book a flight from New York to London
# Agent: Flight Expert
## Task: Process travel query: I need to book a flight from New York to London


Overriding of current TracerProvider is not allowed




# Agent: Flight Expert
## Final Answer: 
To book your flight from New York to London, you would need to decide on specific details such as your preferred departure date, return date if it's a round trip, preferred airline, cabin class, and whether you have any flexibility with your travel plans.

You can book the flight through multiple platforms:
1. Directly through an airline's website, such as American Airlines, British Airways, United Airlines, or Virgin Atlantic, which all operate direct flights between New York (JFK or EWR airports) and London (LHR or LGW airports).
2. Through an online travel agency like Expedia, Orbitz, or Kayak. These platforms allow you to compare prices across multiple airlines.
3. Contacting a travel agent who can provide personalized service and assist you with your travel itinerary.
4. Visit a local airline ticket office if you prefer face-to-face assistance.

During the booking process, you'll be prompted to enter passenger details, select seats, and c

Overriding of current TracerProvider is not allowed




# Agent: Mutual Fund Expert
## Final Answer: 
To provide a satisfactory resolution to the query, it is important to note that determining the "best" mutual funds to invest in greatly depends on individual investment goals, risk tolerance, time horizon, and the current economic environment. Mutual funds can range from low-risk money market funds to high-risk sector-specific funds. Here are some steps to help you identify the best mutual funds for your investment objectives:

1. Investment Goals: Define what you are investing for. Is it retirement, education, a home purchase, or another long-term goal? This will inform the choice of funds that are aligned with your time horizon.

2. Risk Tolerance: Understand your risk appetite. Are you conservative, moderate, or aggressive in terms of risk-taking? Higher risk often comes with the potential for higher returns but also higher volatility and potential losses.

3. Asset Allocation: Based on risk tolerance and investment goals, decide on t

Overriding of current TracerProvider is not allowed




# Agent: General Travel Assistant
## Final Answer: 
To plan a vacation in Europe, you'll need to consider several key points such as destinations, duration, budget, interests, and the type of experiences you're looking to have. Here's a comprehensive plan to get you started:

**1. Destination Selection:**
   - Identify your top countries of interest. Popular choices include Italy for its rich history and cuisine, France for its art and architecture, Spain for vibrant culture and beaches, and Germany for its castles and beer gardens.
   - Research each potential country's top cities and attractions. For example, consider Rome, Paris, Barcelona, and Berlin.

**2. Budget Planning:**
   - Determine your overall budget for the trip, accounting for flights, accommodations, transportation within Europe, food, activities, and incidental expenses.
   - Consider traveling in the shoulder season (spring or fall) for more affordable rates and fewer crowds.

**3. Accommodations:**
   - Decide on 

Overriding of current TracerProvider is not allowed




# Agent: Stock Market Expert
## Final Answer: 
Deciding whether to invest in technology stocks requires a nuanced consideration of various factors that can influence the performance of this sector. 

- **Market Conditions**: Assessing the current market conditions is crucial. Periods of economic growth and innovation typically benefit technology companies. However, during economic downturns or market corrections, tech stocks can experience significant volatility due to their higher growth expectations.

- **Interest Rates and Monetary Policy**: Rising interest rates can reduce the present value of future earnings, which is especially relevant for growth stocks like technology companies that are expected to generate more of their cash flow in the future.

- **Sector Fundamentals**: Investigate the fundamentals of the tech sector, including revenue growth, profitability, competition, and innovation. Companies with strong fundamentals may be better positioned to withstand market downtur